---
title: Python Decorator 装饰器 wrapper
tags: 小书匠,python,built-in,decorater,decorator|wrapper|装饰器,functools,functools.wraps
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# Python Decorator 装饰器 wrapper

## 什么是装饰器

* 装饰器的本质是高阶函数（接受函数为参数的函数）

```
@b
def a(i):
    pass
```

上面的写法实际上是下面写法的语法糖，两者是等价的。

```
a = b(a)
```

只要抓住这一点，许多有关装饰器看起来很奇怪的特性也就能理解了

## 什么时候使用装饰器

一般在两种情况下可以使用装饰器：

1. 需要修改一些函数的行为。如重试、记时
2. 为函数运行提供环境，如添加缓存。（stateful decorator）

## 无参数装饰器

### 实例

#### 例：函数调用时显示函数名

```
def debug(func):
    def wrapper(*arg, **kwargs):
        print("I am function： {}()".format(func.__name__))
        return func(*arg, **kwargs)
    return wrapper

@debug
def test(name='ed'):
    print('hello, I am', name)

# 这实际上相当于 test = debug(test)

if __name__=="__main__":
    test()
    test('yuki')

```

*   注意： `func.__name__` 会返回函数的名字。

#### 例：为函数添加缓存

*   可以利用装饰器和闭包来给函数添加缓存功能。
*   由于装饰器只在函数定义时会运行一次。因此
    *   不同函数的装饰器不会中的变量不会冲突。因为定义时会重新运行一次。
    *   闭包给函数提供了环境，保证每次运行的结果都能保存在环境中
    *   这种特点非常适合给函数添加计数器、计时器等功能

```
def memo(func):
    cache = {} # 注意这句只会运行一次！相当于初始化环境变量，这和python的运行机制有关，可以查看另一篇与装饰器相关的笔记
    def wrapper(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrapper

@memo # 利用装饰器的方式调用，相当于fibonacci = memo(fibonacci)
def fibonacci(n, cache=None):
    if n<=1:
        return 1
    else:
        return fibonacci(n -1) + fibonacci(n-2)

fibonacci(50)

```

#### 例：计时器

In [1]:
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        ret = func(*args, **kwargs)
        time_elapsed = time.time() - start
        print('func: {name} used time {h:02d}:{m:02d}:{s:02d}'.format(name=func.__name__,
                                                          h=int(time_elapsed / 3600),
                                                          m=int(time_elapsed / 60 % 60),
                                                          s=int(time_elapsed % 3600)
                                                          ))
        return ret   

## 使用 functools.wrap

我们希望，我们对装饰器的使用是透明的。

In [2]:
def do_twice(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper
    
@do_twice
def print_hello():
    print('hello')

print_hello()

hello
hello


我们现在来输出一下关于 print_hello 的信息

In [3]:
print(print_hello.__name__)
print(print_hello) # 显示是 do_twice 内部的一个函数 wrapper

wrapper
<function do_twice.<locals>.wrapper at 0x10fb10d08>


由于装饰器返回的是 wrapper 函数，因此经过装饰器装饰之后，我们调用的 print_hello 实际上是 wrapper 函数。因此，我们在查看 print_hello 信息的时候实际上返回的是 wrapp 函数的信息。

我们通常希望，对装饰器的使用是透明的，也就是说，在调用 print_hello 时，我们感受不到装饰器的存在，即我们希望在输出关于 print_hello 的信息的时候，得到的不是 wrapper 函数的信息。

可以使用 `functools.wrap` 来解决这个问题。

注意，`functools.wraps` 只需要对最后返回的那个函数是使用。

In [4]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper
    
@do_twice
def print_hello():
    print('hello')

print_hello()

hello
hello


In [5]:
print(print_hello.__name__)
print(print_hello)

print_hello
<function print_hello at 0x10fb106a8>


可以看到，`functools.wraps` 是一个装饰器，它接受函数func作为参数，它的作用实际上就是 func 的一些属性赋值给 wrapper，我们虽然调用的是 wrapper，但是当我们想查函数的一些信息的时候，查看的是 print_hello 的信息。他的实现比较简单，可以看 [这里](https://github.com/python/cpython/blob/5d4cb54800966947db2e86f65fb109c5067076be/Lib/functools.py#L34)

## 为装饰器添加参数

*   在装饰的外层再添加一个函数用来接受参数，并返回装饰器
*   注意这种方法在没有参数的情况下也要用`@retry()` 调用，不能用 `@retry`

#### 例：利用装饰器添加重试功能

```
def retry(attempt=3, captureError=Exception, raiseException=Exception, callback=None):
    '''
    捕捉到指定的异常captureError重试至多attempt次，重试失败时返回raiseException异常，并调用callback函数
    '''
    def wrapper_(func):
        def wrapper(*args, **kwargs):
            i = 0
            while i<attempt:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    i += 1
                    print('retrying %s times...'%i)
            else:
                if callback is not None:
                    callback()
                raise raiseException
        return wrapper

    return wrapper_

@retry(callback=lambda: print('file open error'))
def test():
    f = open('test','w')
    raise Exception('I am exception')

test()

```

*   注意：这个功能可以用python的retrying库代替，一般还是不要造轮子的为好。但是说不定在哪个服务器上跑没有权限安装retrying库呢？

## 使用类作为装饰器

由于装饰器是一个 Callable 对象，而类可以通过定义 `__call__` 方法将类变成 Callable 对象，因此可以使用类来作为装饰器。这样做的一个目的是为了使用类来保持状态，这在下一节中会有介绍。

In [6]:
import functools
class Counter:
    def __init__(self, func):
        self.count = 0
        self.func = func
        functools.wraps(func)(self)

    def __call__(self, *args, **kwargs):
        self.count += 1
        ret = self.func(*args, **kwargs)
        print("func {} runs {} times".format(self.func.__name__, self.count))
        return ret
    
@Counter
def test():
    print("hello")
    
for i in range(2):
    test()
    
print(test.__name__)
print(test) # 美中不足，test 是一个类，不是一个函数

hello
func test runs 1 times
hello
func test runs 2 times
test


我们来分析上面的代码，由于装饰器的语法是语法糖，因此

```
@Counter
def test():
    pass
```

相当于 

```
test = Counter(test)
```

而 Counter 是一个类，test 是 Counter 类的实例对象。我们希望将 test 当作一个函数使用，也就是希望 Counter 这个类包含 `__call__` 属性。`__call__` 函数的功能实际上和我们之前定义的 `wrapper` 函数相同。

在上面的示例中，我们使用 `functools.wraps` 将 func 的一些属性赋值给 `self`。在函数中，我们是这样使用 `functools.wraps` 的

```
def decorator(func):
    @functools.wraps(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
    return wrapper
```

由于 `@functools.wraps` 是语法糖，它实际上相当于 `wrapper = functools.wraps(func)(wrapper)`

仿照这种用法，我们在类中应该像下面这样调用 `self = functools.wraps(func)(self)`，由于 `functools.wraps` 会直接修改 `self`，因此我们也不需要将结果赋值给 self，因此用 `functools.wraps(func)(self)` 即可。

美中不足的是，由于现在的 test 实际上类的示例，不是一个函数，因此 `print(test)` 输出的信息不是函数，而是类。

### 总结

1. 为了使用类作为装饰器，我们需要定义 `__call__` 方法，同时，我们需要在 `__init__` 中接受 func 参数，这类似于带参数的函数装饰器接受参数。
2. 我们需要定义 `__call__` 同时不需要像函数装饰器那样使用 `wrapper`，`__call__` 的功能就是 `wrapper` 的功能。
3. 我们不能使用 `functools.wraps`，而要使用 `functools.update_wrapper(self, func)`，functools.wraps 作用是返回一个新函数，它实际上是 `functools.update_wrapper` 的一个 partial 版本。
4. `__init__` 中可以定义一些变量用于保持状态。

## 使用装饰器保存状态

使用装饰器的一大原因是装饰器可以用来为函数提供环境，即保持状态，如添加缓存、计算函数调用次数等等。

其中，有一个很大的坑

### 坑！装饰器与可变参数

*   先给出结论：装饰器中使用变量要注意，如果是用来保持状态的变量要用可变参数！不能用不可变参数！（注意：None 也是不可变参数！）
*   原因：因为闭包是浅拷贝，如果是不可变对象，每次调用完成后符号都会被清空，导致错误

### 1. 计数器

##### 错误实例

In [7]:
def simplecounter(func):
    counter = 0
    def wrapper(*args, **kwargs):
        counter += 1
        print('Run funciton %s %s times'%(func.__name__, counter))
        return func(*args, **kwargs)
    return wrapper

@simplecounter
def test():
    print('this is test!')

for i in range(3):
    test()

UnboundLocalError: local variable 'counter' referenced before assignment

*   结果发现错误：

```
UnboundLocalError: local variable 'counter' referenced before assignment

```

*   最后发现引发这个错误的原因是`counter` 是一个int类型，不可变对象。这才会发现这种错误，因此需要将其修改为可变对象，最常见的可变对象就是list了。

##### 修改法1：使用可变对象

*   将 `counter` 定义成可变对象

In [8]:
def simplecounter(func):
    counter = [0]
    def wrapper(*args, **kwargs):
        counter[0] += 1
        print('Run funciton %s %s times'%(func.__name__, counter[0]))
        return func(*args, **kwargs)
    return wrapper

@simplecounter
def test():
    print('this is test!')

for i in range(3):
    test()

Run funciton test 1 times
this is test!
Run funciton test 2 times
this is test!
Run funciton test 3 times
this is test!


##### 修改法2 使用 nonlocal

*   用 `nonlocal` 关键字

In [9]:
def simplecounter(func):
    counter = 0
    def wrapper(*args, **kwargs):
        nonlocal counter
        counter += 1
        print('Run funciton %s %s times'%(func.__name__, counter))
        return func(*args, **kwargs)
    return wrapper

@simplecounter
def test():
    print('this is test!')

for i in range(3):
    test()

Run funciton test 1 times
this is test!
Run funciton test 2 times
this is test!
Run funciton test 3 times
this is test!


##### 修改法三：使用对象装饰器（Best Practice）

In [10]:
class SimpleCounter:
    def __init__(self, func):
        self.func = func
        self.counter = 0
        
    def __call__(self, *args, **kwargs):
        self.counter += 1
        ret = self.func(*args, **kwargs)
        print('Run funciton %s %s times'%(self.func.__name__, self.counter))
        return ret

@SimpleCounter
def test():
    print('this is test!')

for i in range(3):
    test()

this is test!
Run funciton test 1 times
this is test!
Run funciton test 2 times
this is test!
Run funciton test 3 times


#### 限制每10秒调用一次

In [11]:
import time

def limit_time(time_span=10):
    start_time = [0]
    def wrapper(func):
        def wrapper_func(*args, **kwargs):
            now = time.time()
            if now - start_time[0] >= time_span:
                start_time[0] = now
                return func(*args, **kwargs)
            else:
                print("can't run function in a short time! Please wait for {} sec!".format(time_span))
        return wrapper_func

    return wrapper

@limit_time()
def test():
    print('hello world')

for i in range(10):
    test()
    time.sleep(2)

hello world
can't run function in a short time! Please wait for 10 sec!
can't run function in a short time! Please wait for 10 sec!
can't run function in a short time! Please wait for 10 sec!
can't run function in a short time! Please wait for 10 sec!
hello world
can't run function in a short time! Please wait for 10 sec!
can't run function in a short time! Please wait for 10 sec!
can't run function in a short time! Please wait for 10 sec!
can't run function in a short time! Please wait for 10 sec!


# References
- http://localhost:8888/lab/tree/learnPython/Python%20%E8%A3%85%E9%A5%B0%E5%99%A8/Python%20Decorator.ipynb
*   [浅谈Python装饰器-限制函数调用次数的方法 - 简书](https://www.jianshu.com/p/6202e6bce558)
*   [详解Python的装饰器 - 一试就错](https://www.cnblogs.com/cicaday/p/python-decorator.html)
*   [python使用装饰器记录函数执行次数 - 吃不胖的程序猿历程 - CSDN博客](https://blog.csdn.net/qq_31603575/article/details/80011287)
- https://github.com/python/cpython/blob/5d4cb54800966947db2e86f65fb109c5067076be/Lib/functools.py
- [Primer on Python Decorators – Real Python](https://realpython.com/primer-on-python-decorators/)